# Alternative ID Generation Script

This will produce all the various IDs that can be used to lookup in different databases.

In [1]:
# Progress Bar I found on the internet.
# https://github.com/alexanderkuk/log-progress
from progress_bar import log_progress
import re
import sqlite3
import json
from shutil import copy2

In [2]:
WRITE = True
DROP_AND_REMAKE = True

# TSV files containing Pharm-GKB data
PHARMGKB_INTERACTIONS_FILE          = 'tsv_data/pharmgkb/relationships.tsv'
PHARMGKB_CHEMICAL_IDS_FILE          = 'tsv_data/pharmgkb/ids/chemicals.tsv'
PHARMGKB_DRUGS_IDS_FILE             = 'tsv_data/pharmgkb/ids/drugs.tsv'
PHARMGKB_GENES_IDS_FILE             = 'tsv_data/pharmgkb/ids/genes.tsv'
PHARMGKB_PHENOTYPES_IDS_FILE        = 'tsv_data/pharmgkb/ids/phenotypes.tsv'

PHARMGKB_ID_FILES = [
     {"filename":PHARMGKB_CHEMICAL_IDS_FILE},
     {"filename":PHARMGKB_DRUGS_IDS_FILE},
    {"filename":PHARMGKB_GENES_IDS_FILE},
     {"filename":PHARMGKB_PHENOTYPES_IDS_FILE},
]

TABLE_NAME = "alternative_ids"
MESH_VALUE = "mesh"
PHARMGKB_VALUE = "pgkb"
NCBI_VALUE = "ncbi"
VALUES_COLUMN = "vals"
TYPE_COLUMN = "type"
DROP_SQL = 'DROP TABLE IF EXISTS {table};'.format(table = TABLE_NAME)
SCHEMA = '''CREATE TABLE IF NOT EXISTS {table} (
            {col1} VARCHAR UNIQUE,
            {col2} VARCHAR UNIQUE,
            {col3} VARCHAR UNIQUE,
            {col4} VARCHAR,
            {col5} VARCHAR,
            PRIMARY KEY ({col1}, {col2}, {col3})
            );'''.format(
            table = TABLE_NAME, 
            col1=MESH_VALUE,
            col2=PHARMGKB_VALUE,
            col3=NCBI_VALUE,
            col4=VALUES_COLUMN,
            col5=TYPE_COLUMN)
QUERY_UNIQUE_IDS = '''  SELECT DISTINCT geneids1,
                        {type_col}1 FROM interactions
                            union
                        SELECT DISTINCT geneids2,
                        {type_col}2 FROM interactions;'''.format(type_col = TYPE_COLUMN)

PLOS_PMC_DB = 'sqlite_data/data.plos-pmc.sqlite'
ALL_DB = 'sqlite_data/data.all.sqlite'



DELIMITER = "\t"


### Connect to Databases

In [3]:
conn_all = sqlite3.connect(ALL_DB)
cursor_all = conn_all.cursor()

### Create table

In [4]:
if DROP_AND_REMAKE:
    cursor_all.execute(DROP_SQL)
cursor_all.execute(SCHEMA)

    

### Gather Unique IDs into Hash Map
The index is the id, the value is an object.

In [5]:
data = {}

print("Executing SQL query. May take a minute.")
cursor = cursor_all.execute(QUERY_UNIQUE_IDS)
interactions = cursor.fetchall()
print("Query complete")

for row in log_progress(interactions, every=100, name="IDs added"):

    if row == None:
        continue

    
    id = row[0]
    dgr_type = row[1]

    if id not in data:
        data[id] = { TYPE_COLUMN : dgr_type}
        
print("Unique IDs added to data")

Executing SQL query. May take a minute.
Query complete


Unique IDs added to data


## Create lookup table for other values

In [6]:
id_map = {}

for id_file in PHARMGKB_ID_FILES:
    with open(id_file["filename"]) as file:
        try:
            header = None
            linenum = 0
            for line in file:
                linenum+=1
                pgkb = None
                ncbi = None
                mesh = None

                # deliminate the lines
                line = line.strip().split(DELIMITER)

                # Read the headers of the file and assign them to a dictionary {column_name: column_number}
                if linenum == 1:
                    header = {name.strip(): col for col, name in enumerate(line)}
                    print(header)                    
                    continue

                


                # set variables
                pgkb = line[header["PharmGKB Accession Id"]]

                if "NCBI Gene ID" in header:
                    ncbi = line[header["NCBI Gene ID"]]
                if 'External Vocabulary'in header:
                    if len(line) > header['External Vocabulary']: # If a line doesn't have data on the end, it wont be in the delimination
                        external = str(line[header['External Vocabulary']]).replace('"', "") # weird bug where quotes break regex
                        match = re.match('MESH:[0-9A-Za-z]+',external,re.IGNORECASE)
                        if match != None:
                            mesh = match.group(0).upper()

                # fill map   
                values = {PHARMGKB_VALUE: pgkb}                
                if ncbi is not None:
                    values[NCBI_VALUE] = ncbi
                if mesh is not None:
                    values[MESH_VALUE] = mesh
        
                id_map[pgkb] = values
                
                if ncbi is not None:
                    id_map[ncbi] = values
                if mesh is not None:
                    id_map[mesh] = values
                

        except Exception as e:
            print(line)
            raise e

{'PharmGKB Accession Id': 0, 'Name': 1, 'Generic Names': 2, 'Trade Names': 3, 'Brand Mixtures': 4, 'Type': 5, 'Cross-references': 6, 'SMILES': 7, 'InChI': 8, 'Dosing Guideline': 9, 'External Vocabulary': 10, 'Clinical Annotation Count': 11, 'Variant Annotation Count': 12, 'Pathway Count': 13, 'VIP Count': 14, 'Dosing Guideline Sources': 15, 'Top Clinical Annotation Level': 16, 'Top FDA Label Testing Level': 17, 'Top Any Drug Label Testing Level': 18, 'Label Has Dosing Info': 19, 'Has Rx Annotation': 20}
{'PharmGKB Accession Id': 0, 'Name': 1, 'Generic Names': 2, 'Trade Names': 3, 'Brand Mixtures': 4, 'Type': 5, 'Cross-references': 6, 'SMILES': 7, 'InChI': 8, 'Dosing Guideline': 9, 'External Vocabulary': 10, 'Clinical Annotation Count': 11, 'Variant Annotation Count': 12, 'Pathway Count': 13, 'VIP Count': 14, 'Dosing Guideline Sources': 15, 'Top Clinical Annotation Level': 16, 'Top FDA Label Testing Level': 17, 'Top Any Drug Label Testing Level': 18, 'Label Has Dosing Info': 19, 'Has Rx

In [7]:
# Just to test
print(json.dumps(data['1000']))

{"type": "Gene"}


### Go through each ID and find all alternative values

In [8]:


def getType(string):
    if string[:2] in 'PA':
        return PHARMGKB_VALUE
    elif string[:5] in 'MESH:':
        return MESH_VALUE
    else:
        return NCBI_VALUE

count = 0
for value in log_progress(data, every=1000, name="IDs looked up "):

    try:
        ncbi_value = None
        pharmgkb_value = None
        mesh_value = None

        data_values = data.values()
        
        # Get ID type
        if value in id_map:
            map_data = id_map[value]
        

            # Lookup NCBI
            if NCBI_VALUE in map_data:
                ncbi_value = map_data[NCBI_VALUE]
                data[value][NCBI_VALUE] = ncbi_value

            # lookup PharmGKB
            if PHARMGKB_VALUE in map_data:
                pharmgkb_value = map_data[PHARMGKB_VALUE]
                data[value][PHARMGKB_VALUE] = pharmgkb_value

            # Lookup MESH
            if MESH_VALUE in map_data:
                mesh_value = map_data[MESH_VALUE]
                data[value][MESH_VALUE] = mesh_value

        if value not in data_values:
            data[value][getType(value)] = value
            # print(value, data[value])

        # If no alias found
        if len(data[value]) == 2:
            print(value, data[value])
        else:
            count+=1
    except Exception as e:
        print(value, str(e))
    

print("Total with aliases:", count, "/", len(data))

100529141 {'type': 'Gene', 'ncbi': '100529141'}
155060 {'type': 'Gene', 'ncbi': '155060'}
339010 {'type': 'Gene', 'ncbi': '339010'}
441330 {'type': 'Gene', 'ncbi': '441330'}
641590 {'type': 'Gene', 'ncbi': '641590'}
654341 {'type': 'Gene', 'ncbi': '654341'}
85295 {'type': 'Gene', 'ncbi': '85295'}
Haplotype for PA128 {'type': 'VariantLocation', 'ncbi': 'Haplotype for PA128'}
Haplotype for PA166 {'type': 'VariantLocation', 'ncbi': 'Haplotype for PA166'}
Haplotype for PA356 {'type': 'VariantLocation', 'ncbi': 'Haplotype for PA356'}
MESH:C531600 {'type': 'Disease', 'mesh': 'MESH:C531600'}
MESH:C531616 {'type': 'Disease', 'mesh': 'MESH:C531616'}
MESH:C531617 {'type': 'Disease', 'mesh': 'MESH:C531617'}
MESH:C531621 {'type': 'Disease', 'mesh': 'MESH:C531621'}
MESH:C531625 {'type': 'Disease', 'mesh': 'MESH:C531625'}
MESH:C531673 {'type': 'Disease', 'mesh': 'MESH:C531673'}
MESH:C531736 {'type': 'Disease', 'mesh': 'MESH:C531736'}
MESH:C531760 {'type': 'Disease', 'mesh': 'MESH:C531760'}
MESH:C531

MESH:C536211 {'type': 'Disease', 'mesh': 'MESH:C536211'}
MESH:C536221 {'type': 'Disease', 'mesh': 'MESH:C536221'}
MESH:C536223 {'type': 'Disease', 'mesh': 'MESH:C536223'}
MESH:C536227 {'type': 'Disease', 'mesh': 'MESH:C536227'}
MESH:C536228 {'type': 'Disease', 'mesh': 'MESH:C536228'}
MESH:C536231 {'type': 'Disease', 'mesh': 'MESH:C536231'}
MESH:C536237 {'type': 'Disease', 'mesh': 'MESH:C536237'}
MESH:C536254 {'type': 'Disease', 'mesh': 'MESH:C536254'}
MESH:C536273 {'type': 'Disease', 'mesh': 'MESH:C536273'}
MESH:C536282 {'type': 'Disease', 'mesh': 'MESH:C536282'}
MESH:C536293 {'type': 'Disease', 'mesh': 'MESH:C536293'}
MESH:C536294 {'type': 'Disease', 'mesh': 'MESH:C536294'}
MESH:C536305 {'type': 'Disease', 'mesh': 'MESH:C536305'}
MESH:C536311 {'type': 'Disease', 'mesh': 'MESH:C536311'}
MESH:C536317 {'type': 'Disease', 'mesh': 'MESH:C536317'}
MESH:C536321 {'type': 'Disease', 'mesh': 'MESH:C536321'}
MESH:C536329 {'type': 'Disease', 'mesh': 'MESH:C536329'}
MESH:C536330 {'type': 'Disease'

MESH:C537247 {'type': 'Disease', 'mesh': 'MESH:C537247'}
MESH:C537248 {'type': 'Disease', 'mesh': 'MESH:C537248'}
MESH:C537249 {'type': 'Disease', 'mesh': 'MESH:C537249'}
MESH:C537250 {'type': 'Disease', 'mesh': 'MESH:C537250'}
MESH:C537251 {'type': 'Disease', 'mesh': 'MESH:C537251'}
MESH:C537256 {'type': 'Disease', 'mesh': 'MESH:C537256'}
MESH:C537262 {'type': 'Disease', 'mesh': 'MESH:C537262'}
MESH:C537268 {'type': 'Disease', 'mesh': 'MESH:C537268'}
MESH:C537274 {'type': 'Disease', 'mesh': 'MESH:C537274'}
MESH:C537279 {'type': 'Disease', 'mesh': 'MESH:C537279'}
MESH:C537286 {'type': 'Disease', 'mesh': 'MESH:C537286'}
MESH:C537288 {'type': 'Disease', 'mesh': 'MESH:C537288'}
MESH:C537296 {'type': 'Disease', 'mesh': 'MESH:C537296'}
MESH:C537300 {'type': 'Disease', 'mesh': 'MESH:C537300'}
MESH:C537308 {'type': 'Disease', 'mesh': 'MESH:C537308'}
MESH:C537323 {'type': 'Disease', 'mesh': 'MESH:C537323'}
MESH:C537330 {'type': 'Disease', 'mesh': 'MESH:C537330'}
MESH:C537340 {'type': 'Disease'

MESH:C538261 {'type': 'Disease', 'mesh': 'MESH:C538261'}
MESH:C538266 {'type': 'Disease', 'mesh': 'MESH:C538266'}
MESH:C538268 {'type': 'Disease', 'mesh': 'MESH:C538268'}
MESH:C538273 {'type': 'Disease', 'mesh': 'MESH:C538273'}
MESH:C538274 {'type': 'Disease', 'mesh': 'MESH:C538274'}
MESH:C538282 {'type': 'Disease', 'mesh': 'MESH:C538282'}
MESH:C538284 {'type': 'Disease', 'mesh': 'MESH:C538284'}
MESH:C538287 {'type': 'Disease', 'mesh': 'MESH:C538287'}
MESH:C538308 {'type': 'Disease', 'mesh': 'MESH:C538308'}
MESH:C538320 {'type': 'Disease', 'mesh': 'MESH:C538320'}
MESH:C538336 {'type': 'Disease', 'mesh': 'MESH:C538336'}
MESH:C538339 {'type': 'Disease', 'mesh': 'MESH:C538339'}
MESH:C538343 {'type': 'Disease', 'mesh': 'MESH:C538343'}
MESH:C538346 {'type': 'Disease', 'mesh': 'MESH:C538346'}
MESH:C538355 {'type': 'Disease', 'mesh': 'MESH:C538355'}
MESH:C538361 {'type': 'Disease', 'mesh': 'MESH:C538361'}
MESH:C538362 {'type': 'Disease', 'mesh': 'MESH:C538362'}
MESH:C538363 {'type': 'Disease'

MESH:C565948 {'type': 'Disease', 'mesh': 'MESH:C565948'}
MESH:C565957 {'type': 'Disease', 'mesh': 'MESH:C565957'}
MESH:C565973 {'type': 'Disease', 'mesh': 'MESH:C565973'}
MESH:C565974 {'type': 'Disease', 'mesh': 'MESH:C565974'}
MESH:C566021 {'type': 'Disease', 'mesh': 'MESH:C566021'}
MESH:C566023 {'type': 'Disease', 'mesh': 'MESH:C566023'}
MESH:C566112 {'type': 'Disease', 'mesh': 'MESH:C566112'}
MESH:C566121 {'type': 'Disease', 'mesh': 'MESH:C566121'}
MESH:C566136 {'type': 'Disease', 'mesh': 'MESH:C566136'}
MESH:C566162 {'type': 'Disease', 'mesh': 'MESH:C566162'}
MESH:C566178 {'type': 'Disease', 'mesh': 'MESH:C566178'}
MESH:C566273 {'type': 'Disease', 'mesh': 'MESH:C566273'}
MESH:C566288 {'type': 'Disease', 'mesh': 'MESH:C566288'}
MESH:C566291 {'type': 'Disease', 'mesh': 'MESH:C566291'}
MESH:C566331 {'type': 'Disease', 'mesh': 'MESH:C566331'}
MESH:C566333 {'type': 'Disease', 'mesh': 'MESH:C566333'}
MESH:C566366 {'type': 'Disease', 'mesh': 'MESH:C566366'}
MESH:C566367 {'type': 'Disease'

...

PA166170094 {'type': 'Variant', 'pgkb': 'PA166170094'}
PA166170101 {'type': 'Variant', 'pgkb': 'PA166170101'}
PA166170103 {'type': 'Variant', 'pgkb': 'PA166170103'}
PA166170105 {'type': 'Variant', 'pgkb': 'PA166170105'}
PA166170117 {'type': 'Variant', 'pgkb': 'PA166170117'}
PA166170120 {'type': 'Variant', 'pgkb': 'PA166170120'}
PA166170121 {'type': 'Variant', 'pgkb': 'PA166170121'}
PA166170122 {'type': 'Variant', 'pgkb': 'PA166170122'}
PA166170123 {'type': 'Variant', 'pgkb': 'PA166170123'}
PA166170124 {'type': 'Variant', 'pgkb': 'PA166170124'}
PA166170125 {'type': 'Variant', 'pgkb': 'PA166170125'}
PA166170126 {'type': 'Variant', 'pgkb': 'PA166170126'}
PA166170127 {'type': 'Variant', 'pgkb': 'PA166170127'}
PA166170128 {'type': 'Variant', 'pgkb': 'PA166170128'}
PA166170129 {'type': 'Variant', 'pgkb': 'PA166170129'}
PA166170184 {'type': 'Variant', 'pgkb': 'PA166170184'}
PA166170186 {'type': 'Variant', 'pgkb': 'PA166170186'}
PA166170188 {'type': 'Variant', 'pgkb': 'PA166170188'}
PA16617019

PA166176775 {'type': 'Variant', 'pgkb': 'PA166176775'}
PA166176780 {'type': 'Variant', 'pgkb': 'PA166176780'}
PA166176908 {'type': 'Variant', 'pgkb': 'PA166176908'}
PA166176933 {'type': 'Variant', 'pgkb': 'PA166176933'}
PA166176934 {'type': 'Variant', 'pgkb': 'PA166176934'}
PA166176935 {'type': 'Variant', 'pgkb': 'PA166176935'}
PA166176936 {'type': 'Variant', 'pgkb': 'PA166176936'}
PA166176937 {'type': 'Variant', 'pgkb': 'PA166176937'}
PA166176938 {'type': 'Variant', 'pgkb': 'PA166176938'}
PA166176939 {'type': 'Variant', 'pgkb': 'PA166176939'}
PA166176940 {'type': 'Variant', 'pgkb': 'PA166176940'}
PA166176941 {'type': 'Variant', 'pgkb': 'PA166176941'}
PA166176942 {'type': 'Variant', 'pgkb': 'PA166176942'}
PA166176943 {'type': 'Variant', 'pgkb': 'PA166176943'}
PA166176944 {'type': 'Variant', 'pgkb': 'PA166176944'}
PA166176945 {'type': 'Variant', 'pgkb': 'PA166176945'}
PA166176946 {'type': 'Variant', 'pgkb': 'PA166176946'}
PA166176947 {'type': 'Variant', 'pgkb': 'PA166176947'}
PA16617694

PA448081 {'type': 'Drug', 'pgkb': 'PA448081'}
PA448082 {'type': 'Chemical', 'pgkb': 'PA448082'}
PA448084 {'type': 'Chemical', 'pgkb': 'PA448084'}
PA448320 {'type': 'Chemical', 'pgkb': 'PA448320'}
PA448333 {'type': 'Drug', 'pgkb': 'PA448333'}
PA448334 {'type': 'Drug', 'pgkb': 'PA448334'}
PA448360 {'type': 'Chemical', 'pgkb': 'PA448360'}
PA448365 {'type': 'Drug', 'pgkb': 'PA448365'}
PA448368 {'type': 'Chemical', 'pgkb': 'PA448368'}
PA448375 {'type': 'Drug', 'pgkb': 'PA448375'}
PA448382 {'type': 'Chemical', 'pgkb': 'PA448382'}
PA448383 {'type': 'Chemical', 'pgkb': 'PA448383'}
PA448385 {'type': 'Chemical', 'pgkb': 'PA448385'}
PA448388 {'type': 'Chemical', 'pgkb': 'PA448388'}
PA448401 {'type': 'Drug', 'pgkb': 'PA448401'}
PA448404 {'type': 'Chemical', 'pgkb': 'PA448404'}
PA448406 {'type': 'Chemical', 'pgkb': 'PA448406'}
PA448419 {'type': 'Chemical', 'pgkb': 'PA448419'}
PA448428 {'type': 'Drug', 'pgkb': 'PA448428'}
PA448453 {'type': 'Drug', 'pgkb': 'PA448453'}
PA448472 {'type': 'Drug', 'pgkb'

PA450840 {'type': 'Drug', 'pgkb': 'PA450840'}
PA450849 {'type': 'Drug', 'pgkb': 'PA450849'}
PA450850 {'type': 'Drug', 'pgkb': 'PA450850'}
PA450859 {'type': 'Drug', 'pgkb': 'PA450859'}
PA450863 {'type': 'Drug', 'pgkb': 'PA450863'}
PA450864 {'type': 'Drug', 'pgkb': 'PA450864'}
PA450873 {'type': 'Drug', 'pgkb': 'PA450873'}
PA450877 {'type': 'Chemical', 'pgkb': 'PA450877'}
PA450882 {'type': 'Chemical', 'pgkb': 'PA450882'}
PA450897 {'type': 'Drug', 'pgkb': 'PA450897'}
PA450903 {'type': 'Drug', 'pgkb': 'PA450903'}
PA450911 {'type': 'Chemical', 'pgkb': 'PA450911'}
PA450913 {'type': 'Drug', 'pgkb': 'PA450913'}
PA450919 {'type': 'Drug', 'pgkb': 'PA450919'}
PA450921 {'type': 'Chemical', 'pgkb': 'PA450921'}
PA450926 {'type': 'Drug', 'pgkb': 'PA450926'}
PA450931 {'type': 'Drug', 'pgkb': 'PA450931'}
PA450932 {'type': 'Chemical', 'pgkb': 'PA450932'}
PA450935 {'type': 'Chemical', 'pgkb': 'PA450935'}
PA450952 {'type': 'Drug', 'pgkb': 'PA450952'}
PA450954 {'type': 'Drug', 'pgkb': 'PA450954'}
PA450957 {

PA452620 {'type': 'Chemical', 'pgkb': 'PA452620'}
PA452621 {'type': 'Chemical', 'pgkb': 'PA452621'}
PA452629 {'type': 'Chemical', 'pgkb': 'PA452629'}
PA452631 {'type': 'Chemical', 'pgkb': 'PA452631'}
PA452632 {'type': 'Chemical', 'pgkb': 'PA452632'}
PA452633 {'type': 'Chemical', 'pgkb': 'PA452633'}
PA452634 {'type': 'Chemical', 'pgkb': 'PA452634'}
PA452637 {'type': 'Chemical', 'pgkb': 'PA452637'}
PA452638 {'type': 'Chemical', 'pgkb': 'PA452638'}
PA452639 {'type': 'Chemical', 'pgkb': 'PA452639'}
PA452640 {'type': 'Chemical', 'pgkb': 'PA452640'}
PA7001 {'type': 'Drug', 'pgkb': 'PA7001'}
null:PA134946555 {'type': 'VariantLocation', 'ncbi': 'null:PA134946555'}
null:PA33532 {'type': 'VariantLocation', 'ncbi': 'null:PA33532'}
null:PA35845 {'type': 'VariantLocation', 'ncbi': 'null:PA35845'}
null:PA38407 {'type': 'VariantLocation', 'ncbi': 'null:PA38407'}
Total with aliases: 20393 / 29354


In [9]:
data["155060"]

{'type': 'Gene', 'ncbi': '155060'}

### Add data to database

In [10]:
ALIASES_WRITE = '''INSERT OR IGNORE INTO {table} (
                {col1}, {col2}, {col3}, {col4}, {col5} )
                VALUES ( ? , ? , ? , ? , ? );'''.format(
            table = TABLE_NAME, 
            col1=MESH_VALUE,
            col2=PHARMGKB_VALUE,
            col3=NCBI_VALUE,
            col4=VALUES_COLUMN,
            col5=TYPE_COLUMN)

if '' in data:
    del data['']

for key in log_progress(data, every=1000, name="Added to DB"):
    try:
        values = data[key]
        values_str = json.dumps(values)
        values_type = values["type"]
        
        
        mesh_val = None
        pgkb_val = None
        ncbi_val = None
        
        if MESH_VALUE in values:
            mesh_val = values[MESH_VALUE]
        
        if PHARMGKB_VALUE in values:
            pgkb_val = values[PHARMGKB_VALUE]
            
        if NCBI_VALUE in values:
            ncbi_val = values[NCBI_VALUE]

    
        cursor_all.execute(ALIASES_WRITE,(mesh_val, pgkb_val, ncbi_val, values_str, values_type))        

    except Exception as e:
        print(key, values)
        raise e
if WRITE:
    conn_all.commit()
    
print("Data committed")

if WRITE:
    with open("all_aliases_for_mike.json", "w+") as file:
        file.write(json.dumps( data ))
        
print("file written")

Data committed
file written
